In [30]:
import os

In [31]:
%pwd

'/Users/tapankhaladkar/Natural Language Processing/Text-Summarizer/NLP-Text-Summarizer'

In [32]:
#os.chdir('../')

In [33]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    """Configuration for the script."""
    tokenizer_name: Path
    data_path: Path = Path
    root_dir: Path = Path

In [34]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml, create_directories

In [46]:
class ConfigManager:
    """Configuration manager for the text summarization project."""
    
    def __init__(self, config_filepath: str = CONFIG_FILE_PATH, params_filepath: str = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        """Returns the data transformation configuration."""
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            tokenizer_name=config.tokenizer_name,
            data_path=config.data_path,
            root_dir=config.root_dir
        )
        return data_transformation_config

In [36]:
#pip install datasets


In [47]:
import os
import urllib.request as request
import zipfile
from textsummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [50]:
class DataTransformation:
    """Class for transforming data for text summarization."""
    
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True,
        )

        dataset_samsum_pt.save_to_disk(str(Path(self.config.root_dir) / 'samsum_dataset'))


In [51]:
try:
    config = ConfigManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-08-09 20:18:53,139: INFO: YAML file /Users/tapankhaladkar/Natural Language Processing/Text-Summarizer/NLP-Text-Summarizer/config/config.yaml loaded successfully.]
[2025-08-09 20:18:53,142: INFO: YAML file /Users/tapankhaladkar/Natural Language Processing/Text-Summarizer/NLP-Text-Summarizer/params.yaml loaded successfully.]
[2025-08-09 20:18:53,143: INFO: Directory created at: artifacts.]
[2025-08-09 20:18:53,144: INFO: Directory created at: artifacts/data_transformation.]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 98911.43 examples/s] 
